In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from google.colab import files

# Define the tickers for the companies
tickers = ["AAPL", "MSFT", "GOOGL", "NVDA", "NFLX"]

# Create a DataFrame to store all predictions
predictions = pd.DataFrame()
weekly_prices = pd.DataFrame()

# Loop through each ticker symbol
for ticker in tickers:
    # Download historical stock data for the past 1 year
    data = yf.download(tickers=ticker, period="1y")["Adj Close"]

    # Split data into training and testing sets
    training_data_size = int(len(data) * 0.8)
    training_data = data[:training_data_size]
    testing_data = data[training_data_size:]

    # Scale the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_training_data = scaler.fit_transform(training_data.values.reshape(-1, 1))

    # Create sequences for LSTM model
    look_back = 30
    x_train, y_train = [], []
    for i in range(look_back, len(scaled_training_data)):
        x_train.append(scaled_training_data[i-look_back:i, 0])
        y_train.append(scaled_training_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(loss="mean_squared_error", optimizer="adam")

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32)

    # Prepare the testing data for prediction
    scaled_testing_data = scaler.transform(testing_data.values.reshape(-1, 1))
    x_test = []
    for i in range(look_back, len(scaled_testing_data)):
        x_test.append(scaled_testing_data[i-look_back:i, 0])
    x_test = np.array(x_test)

    # Predict the closing price for the next day
    predicted_closing_price = scaler.inverse_transform(model.predict(x_test[-1:].reshape(1, look_back, 1)))[0][0]

    # Append the prediction to the predictions DataFrame
    predictions.loc[ticker, 'Predicted Closing Price'] = predicted_closing_price

    # Extract the last week's data and append to the weekly_prices DataFrame
    week_data = data[-7:]
    weekly_prices[ticker] = week_data.values

    # Plot the stock price history
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data.values, label=ticker)
    plt.xlabel("Date")
    plt.ylabel("Adjusted Closing Price")
    plt.title(f"Stock Price History for {ticker}")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{ticker}_stock_price_history.png")
    plt.close()

# Concatenate the weekly prices with the predictions
predictions = pd.concat([predictions, weekly_prices], axis=1)

# Save the predictions with weekly data to a CSV file

predictions.to_csv('predicted_stock_prices_with_weekly_data.csv')

# Download the CSV file
files.download('predicted_stock_prices_with_weekly_data.csv')

[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
6/6 [==============================] - 7s 71ms/step - loss: 0.2891
Epoch 2/100
6/6 [==============================] - 0s 78ms/step - loss: 0.0588
Epoch 3/100
6/6 [==============================] - 0s 66ms/step - loss: 0.0407
Epoch 4/100
6/6 [==============================] - 0s 71ms/step - loss: 0.0321
Epoch 5/100
6/6 [==============================] - 0s 63ms/step - loss: 0.0285
Epoch 6/100
6/6 [==============================] - 0s 53ms/step - loss: 0.0248
Epoch 7/100
6/6 [==============================] - 0s 64ms/step - loss: 0.0236
Epoch 8/100
6/6 [==============================] - 0s 47ms/step - loss: 0.0231
Epoch 9/100
6/6 [==============================] - 0s 49ms/step - loss: 0.0219
Epoch 10/100
6/6 [==============================] - 0s 52ms/step - loss: 0.0213
Epoch 11/100
6/6 [==============================] - 0s 69ms/step - loss: 0.0206
Epoch 12/100
6/6 [==============================] - 0s 64ms/step - loss: 0.0203
Epoch 13/100
6/6 [==============================]

[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
6/6 [==============================] - 4s 30ms/step - loss: 0.2128
Epoch 2/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0265
Epoch 3/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0169
Epoch 4/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0144
Epoch 5/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0095
Epoch 6/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0075
Epoch 7/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0071
Epoch 8/100
6/6 [==============================] - 0s 30ms/step - loss: 0.0061
Epoch 9/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0057
Epoch 10/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0052
Epoch 11/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0048
Epoch 12/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0046
Epoch 13/100
6/6 [==============================]

[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
6/6 [==============================] - 4s 29ms/step - loss: 0.2287
Epoch 2/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0290
Epoch 3/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0264
Epoch 4/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0204
Epoch 5/100
6/6 [==============================] - 0s 30ms/step - loss: 0.0159
Epoch 6/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0127
Epoch 7/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0128
Epoch 8/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0112
Epoch 9/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0107
Epoch 10/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0100
Epoch 11/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0095
Epoch 12/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0093
Epoch 13/100
6/6 [==============================]

1/1 [==============================] - 2s 2s/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.models import load_model



stock_symbol = "GOOG"  # Adjust to your desired stock
data = yf.download(stock_symbol, period="5mo", interval="1d")

# Define the lookback window for input data (example: 10 timesteps)
# lookback_window = 10

lookback_window = 100



# Select the most recent 'lookback_window' closing prices
x_input = data["Close"][-lookback_window:].values.reshape((1, lookback_window, 1))

# List to store predictions
predictions = []

# Loop to generate 5 predictions
for _ in range(103):
    # Predict the next value
    prediction = model.predict(x_input)
    # Store the predicted value
    predictions.append(prediction[0][0])
    # Update the sequence by removing the first element and appending the new prediction
    x_input = np.append(x_input[0][1:], prediction).reshape((1, lookback_window, 1))

# If data was scaled, reverse scaling for predicted values
scaler = MinMaxScaler()
scaler.fit(data["Close"].values.reshape(-1, 1))
predicted_prices = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

print("Predicted closing prices for the last 5 months:", predicted_prices)



test_data = data[-103:]


pd_test = pd.DataFrame(test_data)
# print (pd_test)

last_close = pd_test["Close"]



mae = np.mean(np.abs(predicted_prices - last_close))

# Calculate the mean of the actual values
mean_actual = np.mean(last_close)

# Calculate relative error
relative_error = mae / mean_actual

# Calculate accuracy as a percentage
accuracy_percentage = (1 - relative_error) * 100

print("Accuracy Percentage:", accuracy_percentage, "%")

[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 53ms/step


1/1 [==============================] - 0s 44ms/step
Predicted closing prices for the last 5 months: [200.23433 211.29164 214.65779 215.75845 215.49046 214.53726 213.32692
 212.08125 210.89505 209.83974 208.9132  207.96237 206.73428 205.05414
 202.92494 200.48566 197.92538 195.423   193.1096  191.02164 189.04352
 186.92746 184.46497 181.65419 178.7147  176.01268 173.97263 172.98355
 173.28662 174.8721  177.44853 180.50867 183.45906 185.76218 187.05511
 187.21092 186.32431 184.64195 182.48288 180.18132 178.05481 176.38106
 175.36975 175.12862 175.6374  176.74643 178.20625 179.72023 181.00507
 181.84459 182.12451 181.84077 181.0839  180.0089  178.8026  177.6544
 176.73177 176.15854 175.99794 176.24297 176.81882 177.59859 178.42865
 179.15854 179.6682  179.88669 179.79959 179.44472 178.89963 178.26479
 177.64622 177.13942 176.81575 176.71281 176.82988 177.12987 177.54738
 178.00182 178.41246 178.7127  178.86037 178.84256 178.67503 178.39656
 178.06053 177.72516 177.44409 177.25829 177.1904